# Implementação do algoritimo completo, sem compromisso com  perfomance

# Imports

In [1]:
import os
import functools
import operator
import math
import random
import sys
import multiprocessing as mp


import pandas as pd
import numpy as np

import fuzzy
import clustering

# Função match

![Match function](./img/match_function.png)

In [ ]:
# %load -s cluster_matching_function 'fuzzy.py'
def cluster_matching_function(weight_matrix,
                              cluster_number,
                              element,
                              prototypes,
                              dissimilarity_matrices):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                cluster_number: int
                    Número do cluster em questão
                element: int
                    Índice do elemento (entre 0 e N-1)
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    # Criando aliases compatíveis com as variáveis da fórmula
    k = cluster_number
    D = dissimilarity_matrices
    p = len(D)
    Gk = prototypes[k]
    l = weight_matrix

    dissimilarities_sum = np.array([dj[element, Gk].sum() for dj in D])

    return np.dot(l[k], dissimilarities_sum)


# Função objetivo

![Objetive function](./img/objective_function.png)

In [ ]:
# %load -s objective_function 'fuzzy.py'
def objective_function(clusters_qtd,
                       elements_qtd,
                       adequacy_criterion,
                       m,
                       weight_matrix,
                       prototypes,
                       dissimilarity_matrices):
    """
        :params: clusters_qtd: int
                    Quantidade total de clusters
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação
                weight_matrix:
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    u = np.power(adequacy_criterion, m) # Resolvendo a exponeciação de u de uma vez só
    l = weight_matrix
    D = dissimilarity_matrices
    K = clusters_qtd
    G = prototypes
    N = elements_qtd
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching
  
    J = np.array([np.array([u[i, k] * match(l, k, i, G, D) for i in range(N)]).sum() 
          for k in range(K)])


    return J.sum()


# Protótipos

![Prototype function](./img/prototype_function.png)

In [ ]:
# %load -s get_prototypes 'fuzzy.py'
def get_prototypes(elements_qtd,
                       q,
                       m,
                       s,
                       cluster_number,
                       adequacy_criterion,
                       dissimilarity_matrices,
                       weight_matrix):
    G = []
    k = cluster_number
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m)
    l = np.power(weight_matrix, s)
    N = elements_qtd
    P = len(D)
    
    while (len(G) != q):
        menor_soma = 999999
        menor_indice = None
        
        for h in range(N): 
            if h in G:
                continue
            
            dists_p = np.array([D[j][:, h] * l[k,j] for j in range(P)]) #shape: NxP
            sums_p = dists_p.sum(axis=0)
            soma = np.dot(u[:, k], sums_p)

            #soma = sum([u[i,k] * sum([l[k,j] * D[j][i,h] for j in range(P)]) for i in  range(N)])

            #print(f"Somas: {soma2} {soma}")

            if soma < menor_soma:
                menor_soma = soma
                menor_indice = h
                 
        G.append(menor_indice)
        
    return G


# Matriz de relevâcia

![Funções de peso](./img/vector_weights_function.png)

In [ ]:
# %load -s compute_relevance_weights 'fuzzy.py'
def compute_relevance_weights(clusters_qtd,
                              dissimilarity_matrices,
                              prototypes,
                              elements_qtd,
                              adequacy_criterion,
                              m):
    """
        :params:
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array of shape K x P

    """

    D = dissimilarity_matrices
    P = len(D)
    G = prototypes
    K = clusters_qtd
    N = elements_qtd
    u = np.power(adequacy_criterion, m)
    l = np.zeros((K, P))

    def match(element, Dh, Gk):
        """
            Função auxiliar para cálculo de match entre um elemento 
            qualquer, os protótipos G de um cluster específico e uma matriz 
            de similaridade específica Dh.
        """

        return Dh[element, Gk].sum()

    for k in range(K):
        # Calculado o somatório do numerador da equação à esquerda da igualdade
        weight_diss_sum1 = np.array([np.array([u[i, k] * match(i, D[h], G[k]) for i in range(N)]).sum()
                            for h in range(P)])

        weight_diss_sum_prod = np.power(weight_diss_sum1.prod(), 1/P)

        for j in range(P):
     
            # Calculado o somatório do denominador da equação à esquerda da igualdade
            weight_diss_sum2 = np.array([u[i, k] * match(i, D[j], G[k])
                                    for i in range(N)]).sum()
            

            # Executando a divisão da fração à esquerda da equação
            l[k, j] = weight_diss_sum_prod / weight_diss_sum2

    return l


# %load -s compute_membership_degree 'fuzzy.py'


# Grau de pertinência

![Fórmula grau de pertinência](./img/membership_degree.png)

In [ ]:
# %load -s compute_membership_degree 'fuzzy.py'
def compute_membership_degree(weight_matrix,
                              prototypes,
                              clusters_qtd,
                              dissimilarity_matrices,
                              elements_qtd,
                              m):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                elements_qtd: int
                    Quantidade de elementos da base de dados
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array NxK

    """
        

    K = clusters_qtd
    G = prototypes
    D = dissimilarity_matrices
    l = weight_matrix
    P = len(D)
    N = elements_qtd
    u = np.zeros((N, K))
    
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching

    def ratio(element, k, h):
        r = match(l, k, element, G, D) / match(l, h, element, G, D)
        #r1 = np.array([l[k,j] * (D[j][element, G[k]].sum()) for j in range(P)]).sum()
        #r2 = np.array([l[h,j] * (D[j][element, G[h]].sum()) for j in range(P)]).sum()
        #r3 = r1/r2
        #print(f"Ratio: {r} {r3}")
        return np.power(r, 1/(m-1))

    for i in range(N):
        for k in range(K):
            outter_sum = np.array([ratio(i, k, h) for h in range(K)]).sum()
            u[i, k] = 1/outter_sum

    return u


# Carregando Matrizes

In [7]:
fac_dis, fou_dis, kar_dis = fuzzy.carregar_matrizes_dissimiliradidades()

# Algoritmo completo
> Partitioning fuzzy K-medoids clustering algorithms with relevance weight for each dissimilarity matrix estimated locally

* Parametros: $K = 10; m = 1.6; T = 150; \epsilon = 10^{−10};$
* Devemos considerar a iniciarlizar do vetor de pesos como sendo 1, já que usamos a equação 9 (MFCMdd-RWL-P)

In [ ]:
# %load -s random_prototypes,executar_treinamento 'fuzzy.py'
def random_prototypes(K, N, q, seed):
    elements = set(range(N))
    protos = []
    random.seed(seed)
    
    for k in range(K):
        protos.append(random.sample(elements, q))
        elements -= set(protos[-1])

    return protos

def executar_treinamento(dissimilarity_matrices,
                       elements_qtd,
                       K=10,
                       m=1.6,
                       T=150,
                       epsilon=10e-10,
                       q=2, 
                       seed=13082020,
                       prototipos = None):

    D = dissimilarity_matrices
    N = elements_qtd
    P = len(D)

    last_lambda = np.ones((K, P))
    last_prototypes = prototipos or random_prototypes(K, N, q, seed)
    last_membership_degree = None
    last_cost = None
    
    assert_relevance_weights_prod_one(last_lambda)

#     print("Passo 0")
#     print("Calculando matriz de adequação inicial (u0)")
    u0 = compute_membership_degree(weight_matrix=last_lambda,
                                   prototypes=last_prototypes,
                                   clusters_qtd=K,
                                   dissimilarity_matrices=dissimilarity_matrices,
                                   elements_qtd=N,
                                   m=m)
    
#     assert_membership_degree_sum_one(u0)

#     print("Calculando função de custo inicial (J0)")
    J0 = objective_function(clusters_qtd=K,
                            elements_qtd=N,
                            adequacy_criterion=u0,
                            m=m,
                            weight_matrix=last_lambda,
                            prototypes=last_prototypes,
                            dissimilarity_matrices=dissimilarity_matrices)
    
    last_membership_degree = u0
    last_cost = J0
    
    for t in range(1, T):
#         print(f"Passo {t}/{T}")
        
#         print(">> Calculando protótipos")
        new_prototypes = [get_prototypes(elements_qtd=N,
                                         q=q,
                                         m=m,
                                         s=1,
                                         cluster_number=k,
                                         adequacy_criterion=last_membership_degree,
                                         dissimilarity_matrices=D,
                                         weight_matrix=last_lambda) for k in range(K)]
        
        #print("new_prototypes.shape", new_prototypes)
        
#         print(">> Calculando matriz de relevâncias")
        new_lambda = compute_relevance_weights(clusters_qtd=K,
                                               dissimilarity_matrices=D,
                                               prototypes=new_prototypes,
                                               elements_qtd=N,
                                               adequacy_criterion=last_membership_degree,
                                               m=m)
        
#         assert_relevance_weights_prod_one(new_lambda)
    
#         print(">> Calculando grau de pertinência")
        new_degree = compute_membership_degree(weight_matrix=new_lambda,
                                               prototypes=new_prototypes,
                                               clusters_qtd=K,
                                               dissimilarity_matrices=dissimilarity_matrices,
                                               elements_qtd=N,
                                               m=m)
    
        
#         assert_membership_degree_sum_one(new_degree)

#         print(">> Calculando função objetivo")
        new_cost = objective_function(clusters_qtd=K,
                                      elements_qtd=N,
                                      adequacy_criterion=new_degree,
                                      m=m,
                                      weight_matrix=new_lambda,
                                      prototypes=new_prototypes,
                                      dissimilarity_matrices=dissimilarity_matrices)

        last_prototypes = new_prototypes
        last_lambda = new_lambda
        last_membership_degree = new_degree
        print(f">> Cost ({seed}): ", new_cost)
        
        if abs(last_cost - new_cost) <= epsilon:
            last_cost = new_cost
            break
    
        last_cost = new_cost
        
    data = {
        "cost":last_cost,
        "membership_degree":last_membership_degree,
        "prototypes":last_prototypes,
        "weight_matrix":last_lambda,
        "times": t,
        "q": q,
        "K":K,
        "m":m,
        "seed": seed,
    }

    return data


# Executando 1x

In [11]:
result = executar_treinamento([fac_dis, fou_dis, kar_dis], 2000, m=1.1, q=2, seed=18082035)

result["cost"]

# fuzzy.export_best_result(result, "data/melhor_resultado_todas.pickle")
# fuzzy.export_fuzzy_partitions_to_csv(result, "data/fuzzy_partitions_todas.csv")

>> Cost:  19534.558421456753
>> Cost:  19289.86375808107
>> Cost:  19223.236397853263
>> Cost:  19138.01160069559
>> Cost:  19120.817673828053
>> Cost:  19120.68364980515
>> Cost:  19120.682802773128
>> Cost:  19120.682796660574
>> Cost:  19120.682796614812
>> Cost:  19120.682796614463


# Executando 100x

In [9]:
TIMES = 100

### TODAS

In [10]:
# Gk = [[1403,1372],[213,1170],[465,1442],[1380,182],[1793,218],[1212,143],[215,995],[1875,416],[840,693],[1643,1811]]

melhor_resultado_todas = fuzzy.executar_algoritmo_varias_vezes([fac_dis, fou_dis, kar_dis], 
                                                                2000, 
                                                                m=1.1,
                                                                report_file = "data/relatorio_varias_execucoes_todas.csv",
                                                                q=2,
                                                                times=TIMES)

fuzzy.export_best_result(melhor_resultado_todas, "data/melhor_resultado_todas.pickle")
fuzzy.export_fuzzy_partitions_to_csv(melhor_resultado_todas, "data/fuzzy_partitions_todas.csv")

melhor_resultado_todas["cost"]

Execução 1/100
>> Cost:  19361.931283225276
Execução 2/100
>> Cost:  19216.84946050788
Execução 3/100
>> Cost:  19418.266638846453
Execução 4/100
>> Cost:  19266.269069275484
Execução 5/100
>> Cost:  19039.149006467207
Execução 6/100
>> Cost:  19262.089666418327
Execução 7/100
>> Cost:  19220.46965808679
Execução 8/100
>> Cost:  18958.234629193725
Execução 9/100
>> Cost:  18962.78641774232
Execução 10/100
>> Cost:  18990.729309877664
Execução 11/100
>> Cost:  19313.21469494188
Execução 12/100
>> Cost:  19200.280164255477
Execução 13/100
>> Cost:  19081.160036454006
Execução 14/100
>> Cost:  19574.9884018176


Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    r

KeyboardInterrupt: 

### FAC

In [ ]:
melhor_resultado_fac = fuzzy.executar_algoritmo_varias_vezes([fac_dis], 2000, times=TIMES)
fuzzy.export_best_result(melhor_resultado_fac, "data/melhor_resultado_fac.pickle")
fuzzy.export_fuzzy_partitions_to_csv(melhor_resultado_fac, "data/fuzzy_partitions_fac.csv")

melhor_resultado_fac["cost"]

### FOU

In [ ]:
melhor_resultado_fou = fuzzy.executar_algoritmo_varias_vezes([fou_dis], 2000, times=TIMES)
fuzzy.export_best_result(melhor_resultado_fou, "data/melhor_resultado_fou.pickle")
fuzzy.export_fuzzy_partitions_to_csv(melhor_resultado_fou, "data/fuzzy_partitions_fou.csv")

melhor_resultado_fou["cost"]

### KAR

In [ ]:
melhor_resultado_kar = fuzzy.executar_algoritmo_varias_vezes([kar_dis], 2000, times=TIMES)
fuzzy.export_best_result(melhor_resultado_kar, "data/melhor_resultado_kar.pickle")
fuzzy.export_fuzzy_partitions_to_csv(melhor_resultado_kar, "data/fuzzy_partitions_kar.csv")

melhor_resultado_kar["cost"]

# Deterinando melhores parâmetros

In [ ]:
# import numpy as np

# qs = list(range(2, 6))
# ms = np.arange(1., 2.1, .1)

# fuzzy.buscar_melhores_parametros(qs, ms, [fac_dis, fou_dis, kar_dis])
# fuzzy.buscar_melhores_parametros(qs, ms, [fac_dis], file_name="data/melhores_parametros_fac.csv"), times= 10)
# fuzzy.buscar_melhores_parametros(qs, ms, [fou_dis], file_name="data/melhores_parametros_fou.csv"), times = 10)
# fuzzy.buscar_melhores_parametros(qs, ms, [kar_dis], file_name="data/melhores_parametros_kar.csv"), times = 10)